<a href="https://colab.research.google.com/github/zzc029498-max/nec-/blob/main/Part_3_Model_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# Import necessary libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split # Added this necessary import

# Import your custom implemented NeuralNet class
try:
    from NeuralNet import NeuralNet
except ImportError:
    print("Error: Could not import NeuralNet class. Please ensure NeuralNet.py is in the directory.")
    NeuralNet = None


# ==============================================================================
# 1. Evaluation metric functions
# ==============================================================================

def calculate_metrics(y_true, y_pred, inverse_transform=True):
    """
    Calculates and returns MSE, MAE, and MAPE.
    Assumes y_true and y_pred are in log space if inverse_transform is True.
    """
    if inverse_transform:
        # Inverse transform (exponential function) to get original prices
        y_true_orig = np.exp(y_true)
        y_pred_orig = np.exp(y_pred)
    else:
        y_true_orig = y_true
        y_pred_orig = y_pred

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true_orig, y_pred_orig)

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true_orig, y_pred_orig)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero and calculate percentage
    mape = np.mean(np.abs((y_true_orig - y_pred_orig) / y_true_orig)) * 100

    return mse, mae, mape

# ==============================================================================
# 2. Data loading
# ==============================================================================

try:
    data = np.load('preprocessed_data.npz')

    # Extract data from the .npz file
    X_train_val = data['X_train_val']
    y_train_val = data['y_train_val']
    X_test = data['X_test']
    y_test = data['y_test']

    # Determine number of input features
    n_features = X_train_val.shape[1]

    print(f"Data loaded successfully. Number of input features: {n_features}")
    print(f"Train/Validation set size: {X_train_val.shape[0]}, Test set size: {X_test.shape[0]}")

except FileNotFoundError:
    print("Error: preprocessed_data.npz file not found. Please ensure Part 1 code was executed.")
    # Create dummy data to allow code execution
    n_features = 12
    X_train_val = np.random.rand(1000, n_features)
    y_train_val = np.random.rand(1000)
    X_test = np.random.rand(200, n_features)
    y_test = np.random.rand(200)

# ==============================================================================
# Part 3.1: BP Hyperparameter comparison and selection (at least 10 combinations)
# ==============================================================================

if NeuralNet is not None:

    # --- Define hyperparameter search space ---
    # Network architecture (n_features: H1: H2: 1)
    architectures = [
        [n_features, 10, 5, 1],
        [n_features, 20, 1],
        [n_features, 50, 20, 1],
        [n_features, 10, 1],
        [n_features, 30, 15, 1],
    ]
    # Learning Rate (η)
    learning_rates = [0.01, 0.1]
    # Momentum (α)
    momenta = [0.0, 0.9]
    # Activation Function
    activation_funcs = ['relu', 'tanh']

    # Fixed parameters
    N_EPOCHS = 500
    VALIDATION_SPLIT = 0.2

    # Generate all combinations
    hyperparam_combinations = []
    for arch in architectures:
        for lr in learning_rates:
            for momentum in momenta:
                for func in activation_funcs:
                    # Filter out some combinations for stability
                    if func == 'relu' and lr > 0.05:
                        continue

                    if func == 'tanh' and momentum == 0.0:
                        continue

                    hyperparam_combinations.append({
                        'arch': arch[1:], # Pass only hidden and output layers
                        'lr': lr,
                        'momentum': momentum,
                        'func': func
                    })

    # Ensure at least 10 combinations
    if len(hyperparam_combinations) < 10:
        hyperparam_combinations = hyperparam_combinations * (10 // len(hyperparam_combinations) + 1)
        hyperparam_combinations = hyperparam_combinations[:10]


    # --- Search Loop ---
    bp_results = []

    print(f"\n--- 3.1 BP Neural Network Hyperparameter Search (Total {len(hyperparam_combinations)} combinations) ---")

    for i, params in enumerate(hyperparam_combinations):

        # Full network architecture: [Input layer size] + [Hidden/Output layer sizes]
        full_arch = [n_features] + params['arch']

        # 1. Train BP Model
        nn = NeuralNet(
            network_architecture=full_arch,
            n_epochs=N_EPOCHS,
            learning_rate=params['lr'],
            momentum=params['momentum'],
            activation_function=params['func'],
            validation_split=VALIDATION_SPLIT
        )

        try:
            # Training (fit method splits X_train_val into train/validation sets internally)
            print(f"\n[{i+1}/{len(hyperparam_combinations)}] Training Arch: {params['arch']}, LR: {params['lr']}, Mom: {params['momentum']}, Func: {params['func']}")
            nn.fit(X_train_val, y_train_val)

            # 2. Predict validation set (Evaluate hyperparameters)
            # Re-split data to ensure correct validation set for prediction check
            _, X_val, _, y_val = train_test_split(
                X_train_val, y_train_val, test_size=VALIDATION_SPLIT, shuffle=True
            )

            y_val_pred_log = nn.predict(X_val).ravel()

            # 3. Calculate metrics
            mse, mae, mape = calculate_metrics(y_val, y_val_pred_log)

            # 4. Store results and loss history
            bp_results.append({
                "Layers": nn.L,
                "Architecture": str(params['arch']),
                "Epochs": N_EPOCHS,
                "LearningRate": params['lr'],
                "Momentum": params['momentum'],
                "Activation": params['func'],
                "MAPE": mape,
                "MAE": mae,
                "MSE": mse,
                "loss_history": nn.loss_epochs(),
                "val_predictions": np.exp(y_val_pred_log),
                "val_true": np.exp(y_val)
            })

        except Exception as e:
            print(f"Training combination {i+1} failed: {e}")
            bp_results.append({
                "Layers": nn.L,
                "Architecture": str(params['arch']),
                "Epochs": N_EPOCHS,
                "LearningRate": params['lr'],
                "Momentum": params['momentum'],
                "Activation": params['func'],
                "MAPE": np.nan, "MAE": np.nan, "MSE": np.nan,
                "loss_history": np.zeros((N_EPOCHS, 2)) * np.nan
            })

    # --- Results Summary and Best Model Selection ---
    bp_df = pd.DataFrame(bp_results).sort_values(by='MAPE', ascending=True).reset_index(drop=True)

    # Report requirement: Table summary of hyperparameter quality
    print("\n\n--- Report Requirement 3.1: BP Neural Network Hyperparameter Search Results (Sorted by MAPE) ---")
    print(bp_df[['Layers', 'Architecture', 'Epochs', 'LearningRate', 'Momentum', 'Activation', 'MAPE', 'MAE', 'MSE']].head(10).to_markdown(index=False, floatfmt=".4f"))

    # Select the best model (lowest MAPE)
    best_bp_params = bp_df.iloc[0]
    print(f"\nBest BP Model Parameters (MAPE: {best_bp_params['MAPE']:.4f}) selected.")


# ==============================================================================
# Part 3.2: Model comparison (BP, MLR-F, BP-F)
# ==============================================================================

# --- 1. Retrain/Evaluate Best BP Model (using all train/validation data) ---
if NeuralNet is not None:
    print("\n--- 3.2 Training Best BP Model and evaluating Test Set ---")

    # Best BP model's full architecture
    best_arch = [n_features] + best_bp_params['arch']

    # Re-instantiate and train the best BP model (using 0.0 validation_split to use all X_train_val)
    best_bp_model = NeuralNet(
        network_architecture=best_arch,
        n_epochs=N_EPOCHS,
        learning_rate=best_bp_params['lr'],
        momentum=best_bp_params['momentum'],
        activation_function=best_bp_params['func'],
        validation_split=0.0 # Use all X_train_val for training
    )

    # Actual training
    try:
        best_bp_model.fit(X_train_val, y_train_val)
        y_test_pred_bp_log = best_bp_model.predict(X_test).ravel()

        # Calculate test set metrics
        metrics_bp = calculate_metrics(y_test, y_test_pred_bp_log)
        y_test_pred_bp_orig = np.exp(y_test_pred_bp_log)

    except Exception as e:
        print(f"Retraining best BP model failed: {e}")
        metrics_bp = (np.nan, np.nan, np.nan)
        y_test_pred_bp_orig = np.zeros_like(y_test) * np.nan


else:
    metrics_bp = (np.nan, np.nan, np.nan)
    y_test_pred_bp_orig = np.zeros_like(y_test) * np.nan


# --- 2. Train MLR-F (Multiple Linear Regression - Scikit-learn) ---
print("\n--- Training MLR-F Model and evaluating Test Set ---")

# Instantiate and train model
mlr_model = LinearRegression()
mlr_model.fit(X_train_val, y_train_val)

# Predict test set
y_test_pred_mlr_log = mlr_model.predict(X_test)

# Calculate test set metrics
metrics_mlr = calculate_metrics(y_test, y_test_pred_mlr_log)
y_test_pred_mlr_orig = np.exp(y_test_pred_mlr_log)


# --- 3. Train BP-F (Framework Neural Network - Scikit-learn MLP) ---
print("\n--- Training BP-F Model and evaluating Test Set ---")

# BP-F Parameters
bp_f_params = {
    'hidden_layer_sizes': (30, 15),
    'activation': 'relu',
    'solver': 'adam',
    'max_iter': N_EPOCHS,
    'random_state': 42,
    'alpha': 0.001, # L2 regularization term
    'learning_rate_init': 0.001,
}
print(f"BP-F Model Description: hidden_layer_sizes={bp_f_params['hidden_layer_sizes']}, activation={bp_f_params['activation']}, max_iter={bp_f_params['max_iter']}")

# Instantiate and train model
bp_f_model = MLPRegressor(**bp_f_params)
# MLPRegressor standardizes data by default, but ours is already preprocessed.
bp_f_model.fit(X_train_val, y_train_val)

# Predict test set
y_test_pred_bpf_log = bp_f_model.predict(X_test)

# Calculate test set metrics
metrics_bpf = calculate_metrics(y_test, y_test_pred_bpf_log)
y_test_pred_bpf_orig = np.exp(y_test_pred_bpf_log)


# ==============================================================================
# 4. Final Comparison Table (Report Requirement 3.2)
# ==============================================================================

results_comparison = pd.DataFrame({
    "Model": ["BP (Student Impl.)", "MLR-F (Linear Regression)", "BP-F (MLP Regressor)"],
    "Parameter Description": [
        f"Arch: {best_bp_params['arch']}, LR: {best_bp_params['lr']}, Mom: {best_bp_params['momentum']}, Func: {best_bp_params['func']}",
        "Scikit-learn Default",
        f"Arch: {bp_f_params['hidden_layer_sizes']}, L2: {bp_f_params['alpha']}"
    ],
    "MSE": [metrics_bp[0], metrics_mlr[0], metrics_bpf[0]],
    "MAE": [metrics_bp[1], metrics_mlr[1], metrics_bpf[1]],
    "MAPE": [metrics_bp[2], metrics_mlr[2], metrics_bpf[2]],
})

print("\n\n--- Report Requirement 3.2: Prediction Quality Comparison on Test Set ---")
print(results_comparison.to_markdown(index=False, floatfmt=".2f"))

# ==============================================================================
# 5. Generate Plotting Data (Report Requirements 3.1 & 3.2)
# ==============================================================================

# --- 5.1 Loss Evolution Plot Data (Report Requirement 3.1) ---
if NeuralNet is not None and len(bp_df) > 0:
    print("\n--- Generating BP Loss Evolution Plot Data ---")

    # Example: Select loss history for Best, Second Best, and Worst models
    plot_models = [
        ("Best BP Model", bp_df.iloc[0]),
        ("Second Best BP Model", bp_df.iloc[1]),
        ("Worst BP Model", bp_df.iloc[-1])
    ]

    # Structure to store data for plotting
    bp_loss_plots_data = []
    for name, row in plot_models:
        bp_loss_plots_data.append({
            "name": f"{name} ({row['Architecture']}-{row['Activation']})",
            "loss_history": row['loss_history']
        })

    # (Use matplotlib to plot this data in your report)


# --- 5.2 Predicted vs True Value Scatter Plot Data (Report Requirements 3.1 & 3.2) ---
print("\n--- Generating Predicted vs True Value Scatter Plot Data ---")

# True values (Inverse transformed)
y_test_orig = np.exp(y_test)

# Data structure for comparison plots
comparison_scatter_data = [
    {
        "model": "BP (Student Impl.) - Best",
        "y_true": y_test_orig,
        "y_pred": y_test_pred_bp_orig,
    },
    {
        "model": "MLR-F (Linear Regression)",
        "y_true": y_test_orig,
        "y_pred": y_test_pred_mlr_orig,
    },
    {
        "model": "BP-F (MLP Regressor)",
        "y_true": y_test_orig,
        "y_pred": y_test_pred_bpf_orig,
    },
]

# (Use matplotlib to plot this data in your report)

# You also need scatter data for 2-3 representative models from the validation set (stored in bp_df)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
